<a href="https://colab.research.google.com/github/Rajat-Kumar-Pandey/MACHINE-LEARNING/blob/main/compleete_drowsy_modal_with_best_accurrecy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout, BatchNormalization
from tensorflow.keras.applications import EfficientNetV2S
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard, Callback
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import logging
import os
import json

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)
logger.info("Script execution started.")

# Constants
SEED = 123
IMAGE_SIZE = 256
CHANNELS = 3
BATCH_SIZE = 32
EPOCHS = 30
N_CLASSES = 2

# Enable mixed precision for faster training on GPUs
policy = tf.keras.mixed_precision.Policy('mixed_float16')
tf.keras.mixed_precision.set_global_policy(policy)
logger.info("Mixed precision policy set to 'mixed_float16'.")

# Check if GPU (CUDA) is available and enable memory growth
physical_devices = tf.config.list_physical_devices('GPU')
if physical_devices:
    logger.info(f"GPU available: {physical_devices}")
    for gpu in physical_devices:
        tf.config.experimental.set_memory_growth(gpu, True)
else:
    logger.warning("No GPU found, using CPU instead.")

# Dataset path
DATASET_PATH = '/content/drive/MyDrive/Drowsiness_Data/Resized_Driver_Drowsiness_Dataset'

# Data augmentation
logger.info("Initializing data augmentation...")
data_gen = ImageDataGenerator(
    rescale=1.0/255.0,
    validation_split=0.2,
    horizontal_flip=True,
    rotation_range=20,
    zoom_range=0.2,
    shear_range=0.2,
    brightness_range=(0.8, 1.2)
)

train_generator = data_gen.flow_from_directory(
    DATASET_PATH,
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='sparse',
    subset='training',
    shuffle=True,
    seed=SEED
)

val_generator = data_gen.flow_from_directory(
    DATASET_PATH,
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='sparse',
    subset='validation',
    shuffle=False,
    seed=SEED
)

# Build EfficientNetV2S model
logger.info("Building EfficientNetV2S model...")
def build_efficientnet_model():
    base_model = EfficientNetV2S(input_shape=(IMAGE_SIZE, IMAGE_SIZE, 3), include_top=False, weights="imagenet")
    base_model.trainable = False  # Freeze the base model

    model = Sequential([
        base_model,
        GlobalAveragePooling2D(),
        BatchNormalization(),
        Dense(1024, activation='relu'),
        Dropout(0.4),
        Dense(512, activation='relu'),
        Dropout(0.3),
        Dense(256, activation='relu'),
        Dropout(0.2),
        Dense(N_CLASSES, activation='softmax', dtype='float32'),
    ])
    return model

cnn_model = build_efficientnet_model()
cnn_model.summary()

# Compile the model
logger.info("Compiling the model...")
cnn_model.compile(
    optimizer=Adam(learning_rate=0.0001),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# Custom callback for saving high-accuracy models
class SaveHighAccuracyModel(Callback):
    def on_epoch_end(self, epoch, logs=None):
        if logs.get('val_accuracy') > 0.98:
            logger.info(f"Validation accuracy exceeded 98% at epoch {epoch + 1}. Saving model...")
            self.model.save(f'efficientnet_high_accuracy_epoch_{epoch + 1}.h5')

# Callbacks
callbacks = [
    ModelCheckpoint('efficientnet_best_model.keras', monitor='val_accuracy', save_best_only=True, verbose=1),
    EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True, verbose=1),
    TensorBoard(log_dir='./logs', histogram_freq=1),
    SaveHighAccuracyModel()
]

# Train the model
logger.info("Starting training...")
history = cnn_model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=EPOCHS,
    callbacks=callbacks,
    verbose=1
)

# Save the final model and training history
cnn_model.save('efficientnet_final_model.h5')
logger.info("Final model saved as 'efficientnet_final_model.h5'.")

with open('training_history.json', 'w') as f:
    json.dump(history.history, f)
logger.info("Training history saved.")
